In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import linecache
from pathlib import Path
from tqdm import tqdm
from sklearn import metrics
import sys

p = Path('.').resolve()
sys.path.append(str(p.parent))

In [2]:
from utils.faiss_utils import *
from utils.data_utils import *

Loading faiss with AVX2 support.


In [3]:
import faiss


def load_XY(basename):
    """
    Load embeddings (X) and possibly the
    labels (Y) of the graph {basename}.
    """
    model_path = Path("/data/models") / basename
    print("Loading data..")
    X, Y = load_data(model_path)
    classes = len(np.unique(Y))
    print("X shape: {}".format(X.shape))
    return X, Y


def centroid_neigh(basename, k_means, X, n=15):
    """
    Find the n-nearest neighbours to k-means
    cluster centroids.
    """
    d = X.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(X)
    D, I = index.search(k_means.centroids, n)
    entities = get_entities_list(basename)
    # find_neighbours(basename, I, entities)
    find_neighbours("itwiki-2013", I, entities)


def find_neighbours(basename, idx, entities):
    """
    Helper function for centroid_neigh.
    """
    urls_file = Path('/data/graphs/') / basename / (basename + '.ids')
    f = urls_file.as_posix()
    for pos, cluster in enumerate(idx):
        print("\x1b[0;35;43m Cluster {} \x1b[0m".format(pos))
        for node in cluster:
            line = entities[node]
            print(linecache.getline(f, line + 1))

Here we use the embeddings learnt on the Italian version of wikipedia from 2013. To learn these embeddings we (randomly) splitted the data into 10 partitions. We now only consider the first partition.

In [4]:
basename = "itwiki-2013_partitioned"
f = "/data/graphs/itwiki-2013/itwiki-2013.ids"

In [5]:
model_path = Path("/data/models") / basename
with (model_path / "entity_names_link_0.json").open() as tf:
    entities_list = json.load(tf)
hf_path = list(model_path.glob("embeddings_link_0*.h5"))[0]
hf = h5py.File(hf_path)
x = hf["embeddings"][:]
# idx = train_search(x)
# _, I = idx.search(x[0].reshape(1, -1), 20)
# for i in I.flatten():
#    line = int(entities_list[i]) + 1
#    print(linecache.getline(f, line))

To measure the quality of the clusters we will use the Silhouette score.

In [6]:
help(metrics.silhouette_score)

Help on function silhouette_score in module sklearn.metrics.cluster.unsupervised:

silhouette_score(X, labels, metric='euclidean', sample_size=None, random_state=None, **kwds)
    Compute the mean Silhouette Coefficient of all samples.
    
    The Silhouette Coefficient is calculated using the mean intra-cluster
    distance (``a``) and the mean nearest-cluster distance (``b``) for each
    sample.  The Silhouette Coefficient for a sample is ``(b - a) / max(a,
    b)``.  To clarify, ``b`` is the distance between a sample and the nearest
    cluster that the sample is not a part of.
    Note that Silhouette Coefficient is only defined if number of labels
    is 2 <= n_labels <= n_samples - 1.
    
    This function returns the mean Silhouette Coefficient over all samples.
    To obtain the values for each sample, use :func:`silhouette_samples`.
    
    The best value is 1 and the worst value is -1. Values near 0 indicate
    overlapping clusters. Negative values generally indicate tha

# 5 clusters

We use k-means with 5 centroids and then calculate the Silhouette score.

In [7]:
itwiki_kmeans = kmeans(x, 5, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Silhouette Coefficient: 0.179


Let's see what are the nodes closest to clusters' centroids..

In [8]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=10)

 Cluster 0 
Park Sung-Wha

Viktor Vasin

Andrej Panavić

Bartosz Bosacki

Davy Schollen

Marc Schneider

Lee Tae-Ho

Jaroslav Šilhavý

Alex Morgan

César Ibáñez

 Cluster 1 
Fort Wayne Pistons 1951-1952

XRR

Albatrellaceae

LUI

Circuito di Zeltweg

Nikon D2H

Lungotevere degli Altoviti

Baciami adesso (Mietta)

Discografia dei Negramaro

Core 2 Quad

 Cluster 2 
Faglia di Cadillac-Larder Lake

Coal Bed Methane

Narcosi da azoto

Ripple mark

Picnoclino

Pompa petrolifera

Associazione Nazionale Istruttori Subacquei

Costa Hamakua

Haifa Chemicals

Nano (prefisso)

 Cluster 3 
Surfin' Bird (singolo)

Scarecrow

Innocence

Copacabana (singolo)

The Mission (colonna sonora)

Big Time

Betrayed

Adrian Edmondson

Chain Reaction

Fight Club

 Cluster 4 
Luigi Valadier

Collegio di Santa Croce

Cattedrale di Santa Maria Assunta (San Severo)

Emo (famiglia)

Pio Panfili

Paolo Pozzo

Giuseppe Zurlo

Chiesa di Nostra Signora del Rosario (Livorno)

Chiesa di Santa Barbara (Livorno)

Domenico 

# 10 clusters

We use k-means with 10 centroids and then calculate the Silhouette score.

In [9]:
itwiki_kmeans = kmeans(x, 10, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Silhouette Coefficient: 0.175


Let's see what are the nodes closest to clusters' centroids..

In [10]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
Adrien Decourcelle

Arthur Arnould

Karl von Abel

Lucien-Anatole Prévost-Paradol

Ernest Picard

 Cluster 1 
A184

Panhard ERC

BTR-40

Force Aérienne Populaire de Benin

BA-10

 Cluster 2 
Pericolosamente insieme

Bolero Extasy

Futureworld - 2000 anni nel futuro

L'esorcista III

La casa dei fantasmi

 Cluster 3 
Carduus acanthoides

Sulpicio Alessandro

Charmahin

Discografia dei Negramaro

Albatrellaceae

 Cluster 4 
Bwejuu

Lingue halmahera-cenderawasih

Bunguran

ISO 3166-2:MG

Limpopo (disambigua)

 Cluster 5 
Trip hop

6 Feet Deep

Echoes, Silence, Patience & Grace

Cannibal Killers Live

It's Nothing

 Cluster 6 
Viktor Vasin

Alex Morgan

Park Sung-Wha

Bartosz Bosacki

Davy Schollen

 Cluster 7 
Collalbrigo

Cantalupo (Imola)

Castello di Buronzo

Monte Poggiolo

Duomo di Sacile

 Cluster 8 
Foresta dell'Alto Palatinato

Ottendorf

Feldkirchen bei Graz

Wolfsberg im Schwarzautal

Wörth an der Lafnitz

 Cluster 9 
Reazione-diffusione

Bioinformatica

Life as We D

# 20 clusters

We use k-means with 20 centroids and then calculate the Silhouette score.

In [11]:
itwiki_kmeans = kmeans(x, 20, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Silhouette Coefficient: 0.235


Let's see what are the nodes closest to clusters' centroids..

In [12]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
Campionati europei di atletica leggera 2012 - 5000 metri piani femminili

Wang Jie

Pallavolo ai Giochi della XXVII Olimpiade

Riccardo Lione

Pallavolo ai Giochi della XXVIII Olimpiade

 Cluster 1 
Saint-Sigismond

Saint-Privat-des-Prés

Sarry (Saona e Loira)

Souspierre

Sainte-Colombe-sur-Seine

 Cluster 2 
Midlothian (disambigua)

Avoca

Aberdeen (disambigua)

Northfield

Plymouth (disambigua)

 Cluster 3 
Premi BAFTA 1954

Premi BAFTA 1953

Il lupo dei mari

La matadora

Margherita della notte

 Cluster 4 
Reazione-diffusione

Bioinformatica

Evoluzione chimica

Dominio della frequenza

Cella primitiva

 Cluster 5 
Strada statale 62 della Cisa

Passo del Lagastrello

Dialetto della Lunigiana

Strada statale 445 della Garfagnana

Savena

 Cluster 6 
Levent Topsakal

Petko Lazarov

Chuck Mrazovich

Éric Beugnot

Gonzalo Sagi-Vela

 Cluster 7 
20451 Galeotti

6585 O'Keefe

4401 Aditi

159826 Knapp

William Kwong Yu Yeung

 Cluster 8 
Lucien-Anatole Prévost-Paradol

Ernest

# 30 clusters

We use k-means with 30 centroids and then calculate the Silhouette score.

In [13]:
itwiki_kmeans = kmeans(x, 30, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Silhouette Coefficient: 0.230


Let's see what are the nodes closest to clusters' centroids..

In [14]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
RSS Persiana

Operazione Simoom

EUROMARFOR

Special Interrogation Group

Diritti umani in Finlandia

 Cluster 1 
Marysville

Sumner (Washington)

White Oak

Des Moines (disambigua)

Contea di Clermont

 Cluster 2 
Petăr Mihtarski

Convocazioni per il campionato europeo di calcio Under-21 2000

Valenciennes Football Club

Coppa dei Campioni 1987-1988

Supercoppa di Francia 1997

 Cluster 3 
Ottendorf

Foresta dell'Alto Palatinato

Trebnitz

Breitenbach

Bronkow

 Cluster 4 
Composizione della membrana cellulare

Fluoresceina sodica

Codone

Argirofilia

Superscan

 Cluster 5 
Il lupo dei mari

La disperata notte

La matadora

Premi BAFTA 1954

Pinky, la negra bianca

 Cluster 6 
Mildura Grand Tennis International 2011

Surbiton Trophy 2005

Challenger of Santa Clarita

Latrobe City Tennis International 2011

Rising Star Tour 2012

 Cluster 7 
Canberra Challenger 1990 - Doppio

Tallahassee Tennis Challenger 2010

Countrywide Classic 1994

Tallahassee Tennis Challenger 2009



# 50 clusters

We use k-means with 50 centroids and then calculate the Silhouette score.

In [15]:
itwiki_kmeans = kmeans(x, 50, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Silhouette Coefficient: 0.220


Let's see what are the nodes closest to clusters' centroids..

In [16]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
Rock am Ring

Trip hop

Rancid

Supergruppo (musica)

Frontman

 Cluster 1 
Brand (Austria)

Wildhaus-Alt St. Johann

Wasserfluh Pass

Luchsingen

Distretto di Weinfelden

 Cluster 2 
Duomo vecchio

Cattedrale di San Giorgio (Ferrara)

Chiesa di San Teodoro (Genova)

Andrea Vicentino

Villa di Poggioreale

 Cluster 3 
Coripe

Ribera Alta (comarca)

Las Tres Villas

Benferri

Fuerte del Rey

 Cluster 4 
Lista di classi di prestigio

Vecna

Carlo Bocchio

Impero (Warhammer)

Tymora

 Cluster 5 
Oscar Apfel

Crepuscolo d'amore (film 1928)

J. Gordon Edwards

John Francis Dillon

Triangle Film Corporation

 Cluster 6 
Special Interrogation Group

Conferenza di Québec (1943)

EUROMARFOR

Crimini nazisti contro i prigionieri di guerra sovietici

Prima battaglia di Char'kov

 Cluster 7 
Bernard Magnin

Levent Topsakal

Gonzalo Sagi-Vela

Adolfo Lubnicki

Kathy Foster

 Cluster 8 
Batote

Sham Chaurasi

Bassi Pathana

Sanawad

Vaijapur

 Cluster 9 
The Serpent's Egg

Fever in the F

# Map score

We will now compute the Mean Average Precision score on the embeddings obtained (considering only out nodes).

In [17]:
from measure_map import map_score

help(map_score)

Help on function map_score in module measure_map:

map_score(X, nodes, ind, neigh_num=50)
    Compute the map score of the given embedding.
    If the number of neighbours of the current node
    is bigger than the one given as input, returns
    the current node as an outlier.
    Input:
        - X (np.array), embeddings
        - nodes (list[list]), neighbours of each node
        - ind (faiss index), index used to compute L2
                            distances for the embeddings
        - neigh_num (int), number of neighbours considered
    Output:
        - score (float), map score
        - outliers (list)
        - singleton, number of singleton nodes



In [18]:
help(nodes_from_ascii)

Help on function nodes_from_ascii in module utils.data_utils:

nodes_from_ascii(basename, in_nodes=False)
    Read nodes from ascii file.
    Input:
        - basename (str), name of the graph
        - in_nodes (bool), if True return in_nodes
    Output:
        nodes (list), list of out_nodes
                    (in_nodes) if in_nodes=True



In [19]:
out_nodes = nodes_from_ascii("itwiki-2013")

1016867 vertices
reading..


100%|██████████| 1016867/1016867 [00:11<00:00, 89719.72it/s]

Found 8215 singleton nodes


In [20]:
entities_list = [int(i) for i in entities_list]

In [21]:
nodes = [out_nodes[i] for i in entities_list]

We need to retrieve the original ids and 

In [22]:
e_list_array = np.array(entities_list)

for i in nodes[0]:
    print(i, np.where(e_list_array == i)[0])

new_nodes = [list() for _ in nodes]
for pos, neigh in tqdm(enumerate(nodes)):
    for n in neigh:
        temp = np.where(e_list_array == n)[0]
        if len(temp) > 0:
            new_nodes[pos].append(temp[0])

289638 []
375342 []
375343 []
375347 []
378815 []
388620 []
388941 [70192]
389964 []
396900 []
522319 []
532258 []
760961 []
861289 []
867371 []
867398 []


101618it [02:20, 723.79it/s]


In [23]:
idx = train_search(x)
score, a, b = map_score(x, new_nodes, idx)
score / len(x)

Index trained: True
Index total: 101618


0.023499976674571738

This isn't the exact map score, since we are not considering the outliers (nodes with more than 50 neighbours) and singletons (nodes with no neighbours).

How many outliers do we have?

In [24]:
len(a)

658

How many singletons?

In [25]:
b

28900

The issue on having so many singletons depends on the fact that partitioning the nodes, some neighbours nodes will end up in different partitions.



Just for curiosity, how many nodes in this partition have more than 1 neighbour?

In [26]:
count = 0
for i in new_nodes:
    if len(i) > 1:
        count += 1
count

48383

We can compute now the MAP score:

In [27]:
score / (len(x) - len(a) - b)

0.03313933707627853

## Compute silhouette score on all partitions

In [28]:
def load_XY(basename):
    """
    Load embeddings (X) and possibly the
    labels (Y) of the graph {basename}.
    """
    model_path = Path("/data/models") / basename
    print("Loading data..")
    X, Y = load_data(model_path)
    classes = len(np.unique(Y))
    print("X shape: {}".format(X.shape))
    return X, Y


def iter_embeddings(model_path, h5=True):
    """
    updated version of iter_partitions
    NOTICE: returns objects NOT ordered
    """
    if h5:
        for h5_file in model_path.glob('embeddings_link*.h5'):
            yield h5_file
    else:
        for json_file in model_path.glob('entity_names_link_*.json'):
            yield json_file


def load_data(model_path):
    """
    Load data saved in model_path.
    Input:
        - model_path (Path)
    Output:
        - X (np.array), embeddings
        - Y (np.array), labels (if possible)
                        otherwise array of zeros.
    """
    x_arrays = []
    y_arrays = []
    for partition in iter_embeddings(model_path):
        h5f = h5py.File(partition, 'r')
        X = h5f["embeddings"][:]
        x_arrays.append(X)
        try:
            Y = h5f["labels"][:]
            y_arrays.append(Y)
        except KeyError:
            print("Labels not defined")
    if len(y_arrays) > 0:
        X = np.vstack(x_arrays)
        Y = np.hstack(y_arrays)
        return X, Y
    else:
        X = np.vstack(x_arrays)
        Y = np.zeros(len(X))
        return X, Y

In [29]:
model_path = Path("/data/models") / basename
for i in iter_embeddings(model_path):
    print(i)

/data/models/itwiki-2013_partitioned/embeddings_link_0.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_2.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_1.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_8.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_3.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_5.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_9.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_7.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_4.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_6.v200.h5


### That's why it doesn't work!

In [30]:
embs = []
for i in iter_embeddings(model_path):
    embs.append(i)
    
for i in sorted(embs):
    print(i)

/data/models/itwiki-2013_partitioned/embeddings_link_0.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_1.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_2.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_3.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_4.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_5.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_6.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_7.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_8.v200.h5
/data/models/itwiki-2013_partitioned/embeddings_link_9.v200.h5


In [31]:
def iter_embeddings(model_path, h5=True):
    """
    updated version of iter_partitions
    NOTICE: returns objects NOT ordered
    """
    temp = []
    if h5:
        for h5_file in model_path.glob('embeddings_link*.h5'):
            temp.append(h5_file)
    else:
        for json_file in model_path.glob('entity_names_link_*.json'):
            temp.append(json_file)
    for i in sorted(temp):
        yield i


In [32]:
x, y = load_XY(basename)

Loading data..
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
X shape: (1016179, 64)


In [33]:
for i in iter_embeddings(model_path, h5=False):
    print(i)

/data/models/itwiki-2013_partitioned/entity_names_link_0.json
/data/models/itwiki-2013_partitioned/entity_names_link_1.json
/data/models/itwiki-2013_partitioned/entity_names_link_2.json
/data/models/itwiki-2013_partitioned/entity_names_link_3.json
/data/models/itwiki-2013_partitioned/entity_names_link_4.json
/data/models/itwiki-2013_partitioned/entity_names_link_5.json
/data/models/itwiki-2013_partitioned/entity_names_link_6.json
/data/models/itwiki-2013_partitioned/entity_names_link_7.json
/data/models/itwiki-2013_partitioned/entity_names_link_8.json
/data/models/itwiki-2013_partitioned/entity_names_link_9.json


In [34]:
entities = []
for i in iter_embeddings(model_path, h5=False):
    entities.append(i)
    
entities

[PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_0.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_1.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_2.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_3.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_4.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_5.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_6.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_7.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_8.json'),
 PosixPath('/data/models/itwiki-2013_partitioned/entity_names_link_9.json')]

In [35]:
entities_list = []
for e in entities:
    with e.open() as f:
        entities_list += json.load(f)

entities_list[:10]

['375341',
 '664430',
 '728407',
 '16627',
 '716009',
 '807080',
 '1011428',
 '230978',
 '426862',
 '381918']

In [36]:
itwiki_kmeans = kmeans(x, 5, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
#print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

In [37]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=10)

 Cluster 0 
Carlo De Lellis

Collegio di Santa Croce

Chiesa della Madonna delle Grazie (Monterotondo)

Agesilao Milano

Emo (famiglia)

Palazzo Gallavresi

Scudo pontificio

Amedeo Lavy

Palazzo Ottolenghi

Catasto gregoriano

 Cluster 1 
It's Not Unusual

Surfin' Bird (singolo)

American Beauty (compilation)

Larry Groupé

Beat Street

Freak

The Prestige: Original Score

Bad Company

Mandy (Barry Manilow)

Top Gun (colonna sonora)

 Cluster 2 
Andrija Kaludjerović

Park Sung-Wha

Karina LeBlanc

Rasim Tagirbekov

Hlompho Kekana

Lesław Ćmikiewicz

Sjarhej Harlukovič

Ihar Šytaŭ

Igor Vrablic

Jan Jílek

 Cluster 3 
Resilienza (biologia)

Ripple mark

Cicli oceanici

Acquaponica

Eluviazione

Blue Hole

Profilo pedologico

Ciclotema

Corsa agli armamenti evolutiva

Aeroplancton

 Cluster 4 
Copa Quito Tenis y Golf Club 2011 - Singolare ragazze

International Tournament of Salsomaggiore 2011 - Singolare ragazze

Panaceo ITF Junior Cup 2011 - Singolare ragazze

307 (numero)

Circuito d

# 10 clusters

We use k-means with 10 centroids and then calculate the Silhouette score.

In [38]:
itwiki_kmeans = kmeans(x, 10, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
#print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Let's see what are the nodes closest to clusters' centroids..

In [39]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
Mario Monti (scrittore)

Christiane Rochefort

Isabella Quarantotti

L'ultimo nastro di Krapp

Colloqui con il professor Y

 Cluster 1 
Waltersdorf (disambigua)

Rosenau

Dobra

Pinkafeld

Merzdorf (disambigua)

 Cluster 2 
Cuba libre - La notte del giudizio

Brian Transeau

Vanilla Ice

Phantom Planet

Garage Days

 Cluster 3 
Luc d'Achery

Daniel Papebroch

Jean Baptiste Cotelier

Pierre Pithou

Heribert Rosweyde

 Cluster 4 
The Alesha Show

Palazzo in via Nilo

Samsung Galaxy Tab 2

Atenione

Pentachaetinae

 Cluster 5 
Andrija Kaludjerović

Karina LeBlanc

Rasim Tagirbekov

Marko Simić

Jan Jílek

 Cluster 6 
Val d'Enza

Licciana Nardi

Fosciandora

Dialetto della Lunigiana

Collagna

 Cluster 7 
Lista degli stati per numero di soldati

Pappagallo verde

Tentera Laut Diraja Malaysia

Type 62

Forze armate libiche

 Cluster 8 
Bangalore Challenger 1993 - Singolare

Nagoya Challenger 1990 - Singolare

Ponte Vedra Challenger 1993

Campos Challenger 1990 - Doppio

Manaus C

# 20 clusters

We use k-means with 20 centroids and then calculate the Silhouette score.

In [40]:
itwiki_kmeans = kmeans(x, 20, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
#print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Let's see what are the nodes closest to clusters' centroids..

In [41]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
Fiat 621

Strapuntino

Massa a pieno carico

Renault TRM 10.000

Fiat 1050 RC.58I

 Cluster 1 
Beatus Rhenanus

Jean Gerson

Andrea Navagero

Prebenda

Antonio Brucioli

 Cluster 2 
Agesiles

Antioco II

Anni 3100 a.C.

782 a.C.

Diga di Ma'rib

 Cluster 3 
Trans-Asian Railway

Choruǧ

Guerra georgiano-abcasa

Lari georgiano

UNFICYP

 Cluster 4 
Pompilio Grandi

Luigi Bodio

Ravachol

Édouard Vaillant

Maria Francesca di Savoia

 Cluster 5 
Mana (videogioco)

Sarah Kerrigan

Codice Veronica

Zero Hour (romanzo)

Halifax (videogiochi)

 Cluster 6 
Greatest Hits (disambigua)

Trip hop

The Jesus Lizard

Don't Touch Me There

Arc Angel

 Cluster 7 
Paroxyclaenidae

Estrildidae

Cycadophyta

Parco nazionale di Marojejy

Cichlidae

 Cluster 8 
Sillaro

Val d'Enza

Dialetto della Lunigiana

Strada statale 62 della Cisa

Felonica

 Cluster 9 
Area superficiale accessibile

Reazione-diffusione

Implicazioni della nanotecnologia sulla salute

Collasso

Meccanosintesi

 Cluster 10 


# 30 clusters

We use k-means with 30 centroids and then calculate the Silhouette score.

In [42]:
itwiki_kmeans = kmeans(x, 30, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
#print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Let's see what are the nodes closest to clusters' centroids..

In [43]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
Contea di Windham (Vermont)

Contea di Somerset (Maine)

Contea di Otsego (Michigan)

Clarendon

Contea di Caledonia

 Cluster 1 
Kalat

Larkana

Distretti del Bhutan

Lingua bishnupriya

Distretto di Trongsa

 Cluster 2 
Nagoya Challenger 1990 - Singolare

Knokke Challenger 1990 - Singolare

Manaus Challenger 1990 - Singolare

Bogotà Challenger 1990 - Doppio

West of England Challenger 1990 - Doppio

 Cluster 3 
Gary Williams (cestista)

Diana Dilova

Krzysztof Fikiel

Reggie Jackson (cestista)

Katalin Szuchy

 Cluster 4 
Sillaro

Bore

Secchia

Strada statale 62 della Cisa

Dialetto della Lunigiana

 Cluster 5 
Source routing (token ring)

Brachystegia tamarindoides

Celemantia

Baggiolara

Rex Hughes

 Cluster 6 
Tom Arnold

James Franco

Hilary Swank

Aaron Eckhart

Mark Ruffalo

 Cluster 7 
Dommartin (Nièvre)

Varennes-sur-Usson

Saint-Hilaire (Allier)

Lapeyrouse (Puy-de-Dôme)

Art-sur-Meurthe

 Cluster 8 
La ragazza di Boemia (film 1936)

L'uomo che prende gli schia

# 50 clusters

We use k-means with 50 centroids and then calculate the Silhouette score.

In [44]:
itwiki_kmeans = kmeans(x, 50, niter=100)
D, I = itwiki_kmeans.index.search(x, 1)
#print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

Let's see what are the nodes closest to clusters' centroids..

In [45]:
centroid_neigh("itwiki-2013_partitioned", itwiki_kmeans, x, n=5)

 Cluster 0 
Quesada

Monção

Corrales

Macieira

Villagomez

 Cluster 1 
Caterina d'Alessandria

Rosone

Facciata a salienti

Pieve dei Santi Cosma e Damiano di Barbassolo

Chiesa di San Nicola di Mira (Rodi Garganico)

 Cluster 2 
Gaetano Quagliariello

Commissione parlamentare per l'indirizzo generale e la vigilanza dei servizi radiotelevisivi

Stefano Rodotà

Marco Causi

Giancarlo Lombardi

 Cluster 3 
Tonight (album David Bowie)

Exposure (Robert Fripp)

Clear Spot

Darryl Jones

Blood on the Snow

 Cluster 4 
Ōshima

Itoman

Prefettura di Fukui

Kanku-dai

Inaba (provincia)

 Cluster 5 
Amodghata

Chandpur

Eksara

Aliganj

Pichhore

 Cluster 6 
Bathyergidae

Propithecus deckenii coronatus

Estrildidae

Bisonalveus

Eurymylidae

 Cluster 7 
Il secondo sesso

Ottimismo

Judith Butler

Luce Irigaray

Filosofia postmoderna

 Cluster 8 
Rote Raben Vilsbiburg

TV Fischbek Amburgo

Convocazioni al torneo di qualificazione europeo al World Grand Prix di pallavolo femminile 2011

Denso A

In [ ]:
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(x, I.flatten()))

# Hierarchical clustering

In [47]:
arr = np.arange(9)
np.random.choice(arr)

4

In [53]:
class BinaryTree():

    def __init__(self, rootid):
        self.left = None
        self.right = None
        self.rootid = rootid

    def getLeftChild(self):
        return self.left

    def getRightChild(self):
        return self.right

    def setNodeValue(self, value):
        self.rootid = value

    def getNodeValue(self):
        return self.rootid

    def insertRight(self, newNode):
        if self.right == None:
            self.right = BinaryTree(newNode)
        else:
            tree = BinaryTree(newNode)
            tree.right = self.right
            self.right = tree

    def insertLeft(self, newNode):
        if self.left == None:
            self.left = BinaryTree(newNode)
        else:
            tree = BinaryTree(newNode)
            tree.left = self.left
            self.left = tree

In [75]:
def random_cut(points, offset=0):
    n = len(points)
    if n == 1:
        return points[0]

    m = np.random.randint(n - 1)
    x = BinaryTree(offset + m)
    split1 = points[:m + 1]
    split2 = points[m + 1:]
#    print(split1)
#    print(split2)
#    print()
    x.insertLeft(random_cut(split1, offset + m))
    x.insertRight(random_cut(split2, offset + m))
    return x


In [77]:
x = random_cut(arr)

[0 1 2 3 4 5 6 7]
[8]

[0 1 2 3]
[4 5 6 7]

[0 1]
[2 3]

[0]
[1]

[2]
[3]

[4]
[5 6 7]

[5 6]
[7]

[5]
[6]



In [82]:
x.getNodeValue()

7

In [84]:
! pip install binarytree

  Created wheel for binarytree: filename=binarytree-4.1.0-py2.py3-none-any.whl size=14352 sha256=12f71ef2af769ceb776beb7fedc4e08d9ce231b8ff825626df1155657aa4e276
  Stored in directory: /home/user/.cache/pip/wheels/e7/34/cd/5860203749b42608a06732024c920359fdac1d10f95f49dcae
Successfully built binarytree


In [85]:
from binarytree import tree

In [86]:
help(tree)

Help on function tree in module binarytree:

tree(height=3, is_perfect=False)
    Generate a random binary tree and return its root node.
    
    :param height: Height of the tree (default: 3, range: 0 - 9 inclusive).
    :type height: int
    :param is_perfect: If set to True (default: False), a perfect binary tree
        with all levels filled is returned. If set to False, a perfect binary
        tree may still be generated by chance.
    :type is_perfect: bool
    :return: Root node of the binary tree.
    :rtype: binarytree.Node
    :raise binarytree.exceptions.TreeHeightError: If height is invalid.
    
    **Example**:
    
    .. doctest::
    
        >>> from binarytree import tree
        >>>
        >>> root = tree()
        >>>
        >>> root.height
        3
    
    .. doctest::
    
        >>> from binarytree import tree
        >>>
        >>> root = tree(height=5, is_perfect=True)
        >>>
        >>> root.height
        5
        >>> root.is_perfect
        T

In [96]:
from binarytree import Node


def random_cut(points):
    n = len(points)
    if n == 1:
        return Node(points[0])

    m = np.random.randint(n - 1)
    x = Node(points[m])
    split1 = points[:m + 1]
    split2 = points[m + 1:]
    x.left = random_cut(split1)
    x.right = random_cut(split2)
    return x


In [97]:
x = random_cut(arr)

In [98]:
print(x)


    __________3__________
   /                     \
  0______               __6__
 /       \             /     \
0       __2         __5       7
       /   \       /   \     / \
      1     3     4     6   7   8
     / \         / \
    1   2       4   5



In [92]:
arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [100]:
d = 64
gauss = np.random.multivariate_normal(np.zeros(d), np.eye(d))

In [101]:
gauss

array([ 0.23876997, -1.8228633 ,  1.62864437, -1.23238509, -1.15971508,
       -0.92923512,  0.57694128, -0.06970784,  0.06092531,  2.21447783,
       -2.94110439,  1.00753319,  0.75696778,  0.77650249, -0.35451452,
        0.64813642, -0.43828405,  0.17209696,  0.23078505, -0.59898101,
       -0.71286433,  0.68465907, -1.21276082, -0.69864822, -0.02391771,
        0.79088219, -0.53106705, -1.7379892 , -2.04991636,  1.53296547,
       -2.10555832,  0.06378074, -0.68413246, -1.28239089, -0.63548918,
        1.22357936, -1.51027068,  1.41509401,  0.8306732 , -0.17616227,
        0.62221228, -1.01856833, -0.48158305, -0.21863284, -1.72246943,
        0.35327656, -1.07679007,  1.65541307,  0.91721578, -0.43251859,
       -0.8407744 , -1.11278227,  1.75129937, -0.54904848, -1.56063471,
        0.55522052, -1.01805231,  1.08149693, -0.18532451,  0.81385412,
       -0.13972167, -2.95707703,  0.1045107 , -0.20825135])

In [103]:
def projected_random_cut(points):
    
    d = points.shape[1]
    g = np.random.multivariate_normal(np.zeros(d), np.eye(d))
    trans = points.dot(g)
    ind = np.argsort(trans)
    return random_cut(trans[ind])


In [104]:
arr = np.random.rand(5, 16)

In [105]:
x = projected_random_cut(arr)

In [109]:
a = Node("ciao")

NodeValueError: node value must be a number

In [132]:
def entity_id(basename, idx, entities):
    """
    Helper function for centroid_neigh.
    """
    urls_file = Path('/data/graphs/') / basename / (basename + '.ids')
    assert urls_file.exists(), "file {} not found!".format(urls_file)
    f = urls_file.as_posix()
    for node in idx:
        line = entities[node]
        temp = linecache.getline(f, line + 1)
        yield temp


entities = get_entities_list(basename)
x, y = load_XY(basename)
inds = np.random.randint(len(x), size=50)
points = x[inds]
ids = [i for i in entity_id("itwiki-2013", inds, entities)]

Loading data..
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
X shape: (1016179, 64)


In [133]:
ids

['Szorosad\n',
 'Cancro e medicina alternativa\n',
 'Rut Brandt\n',
 'Premio Henri Poincaré\n',
 'Antica diocesi di Västerås\n',
 'Caselle (Morimondo)\n',
 'Ballini (zoologia)\n',
 'Challenge League 2010-2011\n',
 'Valore assoluto\n',
 'Giovanni Tarantini\n',
 'Virginio Mezzanzanica\n',
 'Oszkár Csuvik\n',
 'Arthur Rosson\n',
 'Ureaplasma felinum\n',
 'Kleberite\n',
 'Athletic Club 1965-1966\n',
 'Battaglia di Custoza (1848)\n',
 'Alcide Paolini\n',
 'Gene Shue\n',
 'Pratt Institute\n',
 'Muñoz\n',
 'Wutöschingen\n',
 'Guarini\n',
 'Podhradí (Zlín)\n',
 'Brooke Adams\n',
 'Okhotigone\n',
 'Arno Zerbe\n',
 'Ante Nardelli\n',
 'Adattamento del range chemiotattico\n',
 'Viña del Mar Open 1981 - Doppio\n',
 'Microregione di Rio de Janeiro\n',
 'Il mondo di Patty - La storia più bella\n',
 'Gorilla gorilla\n',
 'Federazione Esploratori Italiani\n',
 "L'orso (Čechov)\n",
 'Jannetto de Tassis\n',
 'Miroslav Gajdůsek\n',
 'Beaverton\n',
 'Fabio Capello\n',
 '389 (disambigua)\n',
 "Museo dell'O

In [170]:
import functools as fn


def printBTree(node, nodeInfo=None, inverted=False, isTop=True):

    # node value string and sub nodes
    stringValue, leftNode, rightNode = nodeInfo(node)

    stringValueWidth = len(stringValue)

    # recurse to sub nodes to obtain line blocks on left and right
    leftTextBlock = [] if not leftNode else printBTree(
        leftNode, nodeInfo, inverted, False
    )

    rightTextBlock = [] if not rightNode else printBTree(
        rightNode, nodeInfo, inverted, False
    )

    # count common and maximum number of sub node lines
    commonLines = min(len(leftTextBlock), len(rightTextBlock))
    subLevelLines = max(len(rightTextBlock), len(leftTextBlock))

    # extend lines on shallower side to get same number of lines on both sides
    leftSubLines = leftTextBlock + [""] * (subLevelLines - len(leftTextBlock))
    rightSubLines = rightTextBlock + [""] * (subLevelLines - len(rightTextBlock))

    # compute location of value or link bar for all left and right sub nodes
    #   * left node's value ends at line's width
    #   * right node's value starts after initial spaces
    leftLineWidths = [len(line) for line in leftSubLines]
    rightLineIndents = [len(line) - len(line.lstrip(" ")) for line in rightSubLines]

    # top line value locations, will be used to determine position of current node & link bars
    firstLeftWidth = (leftLineWidths + [0])[0]
    firstRightIndent = (rightLineIndents + [0])[0]

    # width of sub node link under node value (i.e. with slashes if any)
    # aims to center link bars under the value if value is wide enough
    #
    # ValueLine:    v     vv    vvvvvv   vvvvv
    # LinkLine:    / \   /  \    /  \     / \
    #
    linkSpacing = min(stringValueWidth, 2 - stringValueWidth % 2)
    leftLinkBar = 1 if leftNode else 0
    rightLinkBar = 1 if rightNode else 0
    minLinkWidth = leftLinkBar + linkSpacing + rightLinkBar
    valueOffset = (stringValueWidth - linkSpacing) // 2

    # find optimal position for right side top node
    #   * must allow room for link bars above and between left and right top nodes
    #   * must not overlap lower level nodes on any given line (allow gap of minSpacing)
    #   * can be offset to the left if lower subNodes of right node
    #     have no overlap with subNodes of left node
    minSpacing = 2
    rightNodePosition = fn.reduce(
        lambda r, i: max(r, i[0] + minSpacing + firstRightIndent - i[1]),
        zip(leftLineWidths, rightLineIndents[0:commonLines]),
        firstLeftWidth + minLinkWidth,
    )

    # extend basic link bars (slashes) with underlines to reach left and right
    # top nodes.
    #
    #        vvvvv
    #       __/ \__
    #      L       R
    #
    linkExtraWidth = max(0, rightNodePosition - firstLeftWidth - minLinkWidth)
    rightLinkExtra = linkExtraWidth // 2
    leftLinkExtra = linkExtraWidth - rightLinkExtra

    # build value line taking into account left indent and link bar extension (on left side)
    valueIndent = max(0, firstLeftWidth + leftLinkExtra + leftLinkBar - valueOffset)
    valueLine = " " * max(0, valueIndent) + stringValue
    slash = "\\" if inverted else "/"
    backslash = "/" if inverted else "\\"
    uLine = "¯" if inverted else "_"

    # build left side of link line
    leftLink = "" if not leftNode else (
        " " * firstLeftWidth + uLine * leftLinkExtra + slash
    )

    # build right side of link line (includes blank spaces under top node value)
    rightLinkOffset = linkSpacing + valueOffset * (1 - leftLinkBar)
    rightLink = "" if not rightNode else (
        " " * rightLinkOffset + backslash + uLine * rightLinkExtra
    )

    # full link line (will be empty if there are no sub nodes)
    linkLine = leftLink + rightLink

    # will need to offset left side lines if right side sub nodes extend beyond left margin
    # can happen if left subtree is shorter (in height) than right side subtree
    leftIndentWidth = max(0, firstRightIndent - rightNodePosition)
    leftIndent = " " * leftIndentWidth
    indentedLeftLines = [(leftIndent if line else "") + line for line in leftSubLines]

    # compute distance between left and right sublines based on their value position
    # can be negative if leading spaces need to be removed from right side
    mergeOffsets = [len(line) for line in indentedLeftLines]
    mergeOffsets = [
        leftIndentWidth + rightNodePosition - firstRightIndent - w for w in mergeOffsets
    ]
    mergeOffsets = [p if rightSubLines[i] else 0 for i, p in enumerate(mergeOffsets)]

    # combine left and right lines using computed offsets
    #   * indented left sub lines
    #   * spaces between left and right lines
    #   * right sub line with extra leading blanks removed.
    mergedSubLines = zip(range(len(mergeOffsets)), mergeOffsets, indentedLeftLines)
    mergedSubLines = [(i, p, line + (" " * max(0, p))) for i, p, line in mergedSubLines]
    mergedSubLines = [
        line + rightSubLines[i][max(0, -p):] for i, p, line in mergedSubLines
    ]

    # Assemble final result combining
    #  * node value string
    #  * link line (if any)
    #  * merged lines from left and right sub trees (if any)
    treeLines = [leftIndent + valueLine] + (
        [] if not linkLine else [leftIndent + linkLine]
    ) + mergedSubLines

    # invert final result if requested
    treeLines = reversed(treeLines) if inverted and isTop else treeLines

    # return intermediate tree lines or print final result
    if isTop:
        print("\n".join(treeLines))
    else:
        return treeLines


In [178]:
class Node(object):

    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
    def printTree(self):
        printBTree(self, lambda n:(str(n.value), n.left, n.right))


def random_cut(points, ids):
    n = len(points)
    if n == 1:
        return Node(points[0])

    m = np.random.randint(n - 1)
    x = Node(ids[m])
    split1 = points[:m + 1]
    split2 = points[m + 1:]
    x.left = random_cut(split1, ids[:m + 1])
    x.right = random_cut(split2, ids[m + 1:])
    return x

def projected_random_cut(points, ids):
    
    d = points.shape[1]
    g = np.random.multivariate_normal(np.zeros(d), np.eye(d))
    trans = points.dot(g)
    ind = np.argsort(trans)
    return random_cut(trans[ind], [ids[i] for i in ind])

In [184]:
ids = [i.rstrip() for i in ids]

In [188]:
hierarchy = projected_random_cut(points[:7], ids[:7])
hierarchy.printTree()

                                                Premio Henri Poincaré
                                                         / \
                               Antica diocesi di Västerås   0.41692813888078606
                           _______________/  \_______________
                 Rut Brandt                                  Szorosad
                    /  \                                       /  \
 -0.1369111056666767    Ballini (zoologia)  0.10155031787560133    0.27427757663085656
                               /  \
            Caselle (Morimondo)    0.029743462127947484
                    / \
-0.13544552864758197   -0.006078135201100586


In [203]:
from binarytree import Node


def random_cut(points):
    n = len(points)
    if n == 1:
        return Node(points[0])

    m = np.random.randint(n - 1)
    x = Node(points[m])
    split1 = points[:m + 1]
    split2 = points[m + 1:]
    x.left = random_cut(split1)
    x.right = random_cut(split2)
    return x

def projected_random_cut(points):
    
    d = points.shape[1]
    g = np.random.multivariate_normal(np.zeros(d), np.eye(d))
    trans = points.dot(g)
    ind = np.argsort(trans)
    return random_cut(ind)

In [210]:
hierarchy = projected_random_cut(points[:20])

In [211]:
print(hierarchy)


                  __________________8___
                 /                      \
        ________1___                    _10________________________
       /            \                  /                           \
    __3___          _13_______        10                         ___0________________
   /      \        /          \                                 /                    \
  7       _15     13        ___16                    __________14                  ___9___
 / \     /   \             /     \                  /            \                /       \
7   3   15    1           5       8             ___19___          0       _______17       _18
                         / \                   /        \                /         \     /   \
                        5   16                6         _11             2__         9   18    12
                                             / \       /   \           /   \
                                            6   19    11    

In [207]:
for pos, value in enumerate(ids[:20]):
    print(pos, value)

0 Szorosad
1 Cancro e medicina alternativa
2 Rut Brandt
3 Premio Henri Poincaré
4 Antica diocesi di Västerås
5 Caselle (Morimondo)
6 Ballini (zoologia)
7 Challenge League 2010-2011
8 Valore assoluto
9 Giovanni Tarantini
10 Virginio Mezzanzanica
11 Oszkár Csuvik
12 Arthur Rosson
13 Ureaplasma felinum
14 Kleberite
15 Athletic Club 1965-1966
16 Battaglia di Custoza (1848)
17 Alcide Paolini
18 Gene Shue
19 Pratt Institute
